In [1]:
# Try importing with debug info
import sys
print("Python path:", sys.path)

import llm_utils
print("\nllm_utils location:", llm_utils.__file__)
print("\nContents of llm_utils:", dir(llm_utils))

# Try reloading the module to get latest changes
import importlib
importlib.reload(llm_utils)
print("\nAfter reload - Contents of llm_utils:", dir(llm_utils))

Python path: ['/usr/lib64/python311.zip', '/usr/lib64/python3.11', '/usr/lib64/python3.11/lib-dynload', '', '/proj/sas/ripython/venv_311/lib64/python3.11/site-packages', '/proj/sas/ripython/venv_311/lib/python3.11/site-packages']

llm_utils location: /proj/sas/ripython/LLMClassfication/GitHub/SDoH-LLMClassification/llm_utils.py

Contents of llm_utils: ['AzureOpenAI', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'ast', 'calculate_disagreement_score', 'calculate_multilabel_metrics', 'cohen_kappa_score', 'convert_gold_labels_to_list', 'create_2shot_hard_prompt', 'generate_explanation_for_easy_example', 'generate_explanation_for_hard_example', 'get_prompt_template', 'get_strengthened_prompt_template', 'json', 'np', 'os', 'parse_llm_labels', 'precision_recall_fscore_support', 're', 'run_llm_annotation', 'run_zero_shot_and_score', 'select_easy_examples_automatically', 'select_hard_examples_automatically', 'time']

After reload - Co

In [2]:
import os
import re
import ast
import pickle
import numpy as np
import pandas as pd
from openai import AzureOpenAI
import time
from sdoh_data import load_sdoh_dataset   # ✅ helper module
from llm_utils import (parse_llm_labels, run_llm_annotation, calculate_disagreement_score, run_zero_shot_and_score,select_hard_examples_automatically,
                        generate_explanation_for_hard_example, create_2shot_hard_prompt, select_easy_examples_automatically, generate_explanation_for_easy_example,
                        calculate_multilabel_metrics)
from sklearn.metrics import precision_recall_fscore_support, cohen_kappa_score

### Best Practice for Multi-Label Hard Example Selection:Hamming Loss Approach
1) Handles class imbalance naturally - We can weight by category frequency
2) Provides rich teaching examples - An example that's wrong on multiple categories teaches the model more
3) Standard in multi-label literature - Hamming distance/loss is the go-to metric for multi-label disagreement

In [3]:
categories = ["employment", "housing", "smoking", "alcohol"]

# Example 1: Model misses "alcohol"
gold = ["smoking", "alcohol"]
pred = ["smoking"]
score, errors = calculate_disagreement_score(gold, pred, categories)
print(f"Score: {score}, Errors: {errors}")
# Output: Score: 1, Errors: {'alcohol': 'FN (missed)'}

# Example 2: Model adds "employment" wrongly
gold = ["smoking"]
pred = ["smoking", "employment"]
score, errors = calculate_disagreement_score(gold, pred, categories)
print(f"Score: {score}, Errors: {errors}")
# Output: Score: 1, Errors: {'employment': 'FP (false alarm)'}

# Example 3: Multiple errors
gold = ["smoking", "alcohol"]
pred = ["employment", "housing"]
score, errors = calculate_disagreement_score(gold, pred, categories)
print(f"Score: {score}, Errors: {errors}")
# Output: Score: 4, Errors: {...}

Score: 1, Errors: {'alcohol': 'FN (missed)'}
Score: 1, Errors: {'employment': 'FP (false alarm)'}
Score: 4, Errors: {'employment': 'FP (false alarm)', 'housing': 'FP (false alarm)', 'smoking': 'FN (missed)', 'alcohol': 'FN (missed)'}


In [5]:
# Test run with 10 samples
df = load_sdoh_dataset()
categories = [c for c in df.columns if c != "premise"]
categories
df_gold = pd.read_csv("annotations_with_gold.csv")

test_results = run_zero_shot_and_score(
    df=df_gold,  # Your dataframe with gold labels
    categories=categories,  # Your 10 categories
    deployment_name="gpt-35-turbo",
    sample_size=10  # Small test
)

# Check the results
print(test_results[['premise', 'gold_labels', 'predicted_labels', 'disagreement_score']].head())

Running 0-shot on 10 samples...

Completed! Disagreement score stats:
count    10.0
mean      0.0
std       0.0
min       0.0
25%       0.0
50%       0.0
75%       0.0
max       0.0
Name: disagreement_score, dtype: float64
                                               premise     gold_labels  \
507  The patient is employed in the finance department              []   
70   He also quit smoking cigarettes in 1984, after...     ['smoking']   
131                    He has occasional glass of wine     ['alcohol']   
400  She is presently unemployed, but plans to go o...  ['employment']   
541  The patient smokes about a pack a day for more...     ['smoking']   

    predicted_labels  disagreement_score  
507               []                   0  
70         [smoking]                   0  
131        [alcohol]                   0  
400     [employment]                   0  
541        [smoking]                   0  


***Total 570 rows, from which 50 records are sampled for creating a variety of shots that will be used by LLM to label the test set.***

In [6]:
# =================================================================================
# Step 1: run 0 shot first to create baseline
# =================================================================================
from IPython.display import display, HTML
test_results = run_zero_shot_and_score(
    df=df_gold,  # Your dataframe with gold labels
    categories=categories,  # Your 10 categories
    deployment_name="gpt-35-turbo",
    sample_size=50  
)
pd.set_option('display.max_colwidth', None)
test_results.sort_values("disagreement_score", ascending=False, inplace=True)
# test_results[['premise', 'gold_labels', 'predicted_labels', 'disagreement_score']]
def wrap_text(s, width=60):
    return '<br>'.join([s[i:i+width] for i in range(0, len(s), width)])

test_results_wrapped = test_results.copy()
test_results_wrapped['premise'] = test_results_wrapped['premise'].apply(lambda x: wrap_text(str(x), width=60))
display(HTML(test_results_wrapped[['premise', 'gold_labels', 'predicted_labels', 'disagreement_score', 'errors']].to_html(escape=False)))

Running 0-shot on 50 samples...

Completed! Disagreement score stats:
count    50.000000
mean      0.040000
std       0.197949
min       0.000000
25%       0.000000
50%       0.000000
75%       0.000000
max       1.000000
Name: disagreement_score, dtype: float64


,premise,gold_labels,predicted_labels,disagreement_score,errors
90,He denies any smoking or drug usage.,['smoking'],[],1,{'smoking': 'FN (missed)'}
76,He currently lives with his mother in house for several both in New York and here in Colorado,['housing'],[],1,{'housing': 'FN (missed)'}
131,He has occasional glass of wine,['alcohol'],[alcohol],0,{}
400,"She is presently unemployed, but plans to go on a job interview today with Alaska USA Federal Credit Union",['employment'],[employment],0,{}
541,The patient smokes about a pack a day for more than 25 years,['smoking'],[smoking],0,{}
362,She has a 27 pack year smoking history,['smoking'],[smoking],0,{}
188,He smoked a pack and a half of cigarettes per day for the past 35 years,['smoking'],[smoking],0,{}
29,Denied tobacco/ETOH/illicit drug use.,[],[],0,{}
81,He denies alcohol or drug abuse.,[],[],0,{}
250,"No history of ETOH, Tobacco or illicit drug use",[],[],0,{}


In [9]:
# =================================================================================
# Step 2a: select hard examples  
# =================================================================================
hard_examples = select_hard_examples_automatically(test_results_wrapped, top_n=2)

print("Automatically selected hard examples:\n")
for i, ex in enumerate(hard_examples, 1): # 1 specifies the starting index. by default, enumerate() starts counting from 0
    print(f"Hard Example {i}:") # or if by default starting from 0, i here would be i+1 
    print(f"  Premise: {ex['premise'][:80]}...")
    print(f"  Gold: {ex['gold_labels']}")
    print(f"  Predicted: {ex['predicted_labels']}")
    print(f"  Score: {ex['disagreement_score']}")
    print()

Automatically selected hard examples:

Hard Example 1:
  Premise: He denies any smoking or drug usage....
  Gold: ['smoking']
  Predicted: []
  Score: 1

Hard Example 2:
  Premise: He currently lives with his mother in house for several both<br> in New York and...
  Gold: ['housing']
  Predicted: []
  Score: 1



In [7]:
# ===========================Sanity Check=====================================
# Check what error types we captured in our 50 samples
# Analyze error patterns
# ============================================================================
error_analysis = []
for idx, row in test_results_wrapped[test_results_wrapped['disagreement_score'] > 0].iterrows():
    for cat, error_type in row['errors'].items():
        error_analysis.append({
            'category': cat,
            'error_type': error_type
        })

error_df = pd.DataFrame(error_analysis)
print("Error types captured in 50 samples:")
print(error_df.groupby(['category', 'error_type']).size())

Error types captured in 50 samples:
category  error_type 
housing   FN (missed)    1
smoking   FN (missed)    1
dtype: int64


***Analysis***<br>
Concern 1: 50 Samples - Representative Enough?
    About the 50 Sample Size:
    Let's think statistically:

    HuggingFace data: 500+ samples
    50 samples = 10%
    Paper used 100 from thousands ≈ 5-10%
    Proportion is similar! ✅
Concern 2: are categories representative Enough?
    Rationale:
        The top categories (smoking, alcohol, employment) are well-represented
        The missing 3 are probably very rare in your full 500+ dataset anyway
        Hard examples will teach: "Don't over-predict" (the main problem)
        Paper didn't require ALL categories represented
    Conclusion:
        The 50 sample analysis is solid because:

        Error pattern is clear: FP on negations (the top 2 hard examples with score=3)
        Major categories covered: smoking, alcohol, employment
        Matches paper proportion: 10% of your data
        Rare categories (food, transportation, opioids) might have too few samples anyway

In [9]:
# Check category coverage in your 50 samples
print("Gold label distribution in 50 samples:")
all_categories_in_50 = []
for labels in test_results_wrapped['gold_labels']:
    all_categories_in_50.extend(parse_llm_labels(labels))

from collections import Counter
print(Counter(all_categories_in_50))

Gold label distribution in 50 samples:
Counter({'smoking': 12, 'alcohol': 6, 'employment': 4, 'housing': 1, 'marijuana': 1, 'cocaine': 1, 'drug_use': 1})


In [10]:
# =================================================================================
# Step 2b: Generate explanations for both hard examples
# ================================================================================= 
for i, hard_ex in enumerate(hard_examples, 1):
    print(f"\n{'='*70}")
    print(f"Hard Example {i}:")
    print(f"{'='*70}")
    print(f"Premise: {hard_ex['premise']}")
    print(f"Gold: {hard_ex['gold_labels']}")
    print(f"Predicted: {hard_ex['predicted_labels']}")
    print(f"Score: {hard_ex['disagreement_score']}")
    print(f"\nGenerating explanation...")
    
    explanation = generate_explanation_for_hard_example(
        premise=hard_ex['premise'],
        gold_labels=parse_llm_labels(hard_ex['gold_labels']),
        predicted_labels=hard_ex['predicted_labels'],
        categories=categories,
        deployment_name="gpt-35-turbo"
    )
    
    hard_ex['explanation'] = explanation
    print(f"\nExplanation:")
    print(explanation)
    print()
    
    time.sleep(1)  # Rate limiting
# The Psychology of LLMs:
#     0-Shot (Blind Guessing):
# With Correct Answer (Reflective Mode):
# This is EXACTLY Why 2-Shot Learning Works!
# 0-shot: Model uses shallow pattern matching
# 2-shot with explanations: Model learns to think deeper
# Result: Model starts paying attention to negations!

# 📚 Why This Happens (Research Findings):
# 1. Recency Bias:

# LLMs pay more attention to things closer to the answer
# Your rules are 10+ lines before the premise
# By the time it reads "Does not smoke", Rule 6 is "forgotten"

# 2. Instruction Hierarchy:

# Rule 0 says: "Identify all applicable categories"
# Model thinks: "Find categories!" (action-oriented)
# Negation is exception/constraint (less salient)

# 3. Token-Level Pattern Matching:

# "smoke", "drink", "drug" have strong embeddings → categories
# "Does not" is a modifier (weaker signal)
# GPT-3.5 (non-reasoning model) doesn't do multi-step logic:

# Step 1: Check for negation
# Step 2: If negation, suppress categories
# It just pattern matches! 🤷


Hard Example 1:
Premise: He denies any smoking or drug usage.
Gold: ['smoking']
Predicted: []
Score: 1

Generating explanation...

Explanation:
The model was wrong because it failed to recognize that the phrase "denies any smoking" still indicates the presence of the concept "smoking" in the text, despite the negation. The key phrase "denies any smoking" explicitly references smoking behavior, which should be annotated regardless of negation. The model should follow the rule to annotate all mentioned clinical concepts, even if they are negated, to accurately capture the full scope of relevant information.


Hard Example 2:
Premise: He currently lives with his mother in house for several both<br> in New York and here in Colorado
Gold: ['housing']
Predicted: []
Score: 1

Generating explanation...

Explanation:
The model missed the annotation because it failed to recognize that "lives with his mother in house" indicates a housing-related context. The key phrase "lives with his mother in 

In [17]:
# Create a test prompt
test_premise = "Patient denies smoking or alcohol use."
test_prompt = create_2shot_hard_prompt(hard_examples, categories, test_premise)

print(test_prompt)

You are an expert annotator for clinical text.

Task:
Given a patient note (premise), identify which of the following categories apply:
marijuana, food, cocaine, transportation, housing, drug_use, opioids, smoking, employment, alcohol

Rules:
0) Identify all applicable categories from the list above that apply to the patient note.
Return ONLY a JSON array of applicable categories, with NO explanation.
1) UMBRELLA DRUG RULE: If opioids, marijuana, or cocaine are present, ALSO include "drug_use".
2) EMPLOYMENT: Label ONLY if unemployed, job loss, or work problems. Do NOT label if working, retired, student, or homemaker.
3) HOUSING: Label only if unstable (homeless, shelter, unsafe). Do NOT label for neutral mentions.
4) SUBSTANCE USE: Include if use is stated (current or past). Negations cancel it.
5) FOOD & TRANSPORTATION: Label only if lack/barrier exists.
6) NEGATIONS: If "never", "denies", "none", "no history" or any negative words → do NOT include that category.
7) AMBIGUITY: If spe

how to split data: Use ~200 samples for testing<br>
    - Power Analysis:
        200 samples gives 80% statistical power to detect 10% difference in F1 (α=0.05)
        Sufficient for Cohen's kappa reliability (±0.05 confidence interval)
    - Cost-Benefit:
        200 samples × 3 configs = 600 API calls
        Cost: ~$1-2
        Time: ~10-15 minutes
        Good balance!
    - Category Coverage:
        Major categories (smoking, alcohol, employment): 40-80 samples each ✅
        Rare categories: Still limited but best we can do with your data size
Reserve Data:
    Keeps ~250 samples unused
    Can use later for:
XGBoost training (like the paper)
Validation
Future experiments

📊 Confidence Intervals with 200 Samples:
Metric          95%CI Width Interpretation
F1 Score        ±0.05       Good precisionCohen's 
Kappa           ±0.05       Reliable agreement
Per-category F1 ±0.10-0.15  Acceptable for major categories

In [ ]:
# =================================================================================
# Step 2c: Split your remaining data
# ================================================================================= 
results_50=test_results
remaining_df = df_gold.drop(results_50.index)  # Remove the 50 used for hard examples

# Randomly sample 200 for testing
test_df = remaining_df.sample(n=200, random_state=42)

print(f"Test set size: {len(test_df)}")
print(f"Reserved for future: {len(remaining_df) - len(test_df)}")

# Check category distribution in test set
test_categories = []
for labels in test_df['gold_labels']:
    test_categories.extend(parse_llm_labels(labels))

from collections import Counter
print("\nCategory distribution in test set:")
print(Counter(test_categories))

Test set size: 200
Reserved for future: 320

Category distribution in test set:
Counter({'alcohol': 33, 'smoking': 33, 'drug_use': 7, 'marijuana': 3, 'employment': 3, 'opioids': 2, 'cocaine': 2, 'housing': 1, 'transportation': 1})


In [ ]:
# =================================================================================
# Step 3: First, Select Easy Examples
# Select easy examples from your 50-sample results
# =================================================================================
easy_positive, easy_negative = select_easy_examples_automatically(results_50)

# Check what we got
print("Easy Positive Example (Model got it RIGHT):")
print(f"  Premise: {easy_positive['premise'][:80]}...")
print(f"  Gold: {easy_positive['gold_labels']}")
print(f"  Predicted: {easy_positive['predicted_labels']}")
print(f"  Score: {easy_positive['disagreement_score']}")
print()

print("Easy Negative Example (Model got it RIGHT):")
print(f"  Premise: {easy_negative['premise'][:80]}...")
print(f"  Gold: {easy_negative['gold_labels']}")
print(f"  Predicted: {easy_negative['predicted_labels']}")
print(f"  Score: {easy_negative['disagreement_score']}")

Easy Positive Example (Model got it RIGHT):
  Premise: The patient smokes about a pack a day for more than 25 years...
  Gold: ['smoking']
  Predicted: ['smoking']
  Score: 0

Easy Negative Example (Model got it RIGHT):
  Premise: She has never used illicit drugs...
  Gold: []
  Predicted: []
  Score: 0


In [21]:
print("Generating explanations for easy examples...")
print()

# Easy positive explanation
easy_pos_explanation = generate_explanation_for_easy_example(
    premise=easy_positive['premise'],
    gold_labels=parse_llm_labels(easy_positive['gold_labels']),
    predicted_labels=easy_positive['predicted_labels'],
    categories=categories,
    deployment_name="gpt-35-turbo"
)
easy_positive['explanation'] = easy_pos_explanation

print("Easy Positive Explanation:")
print(easy_pos_explanation)
print()

time.sleep(1)

# Easy negative explanation
easy_neg_explanation = generate_explanation_for_easy_example(
    premise=easy_negative['premise'],
    gold_labels=parse_llm_labels(easy_negative['gold_labels']),
    predicted_labels=easy_negative['predicted_labels'],
    categories=categories,
    deployment_name="gpt-35-turbo"
)
easy_negative['explanation'] = easy_neg_explanation

print("Easy Negative Explanation:")
print(easy_neg_explanation)

Generating explanations for easy examples...

Easy Positive Explanation:
The key phrase in this text that determines the classification as 'smoking' is "smokes about a pack a day for more than 25 years." This answer is correct because it directly mentions the patient's smoking habit, which is a significant risk factor for various health conditions. In similar cases, the model should look for specific mentions of smoking behavior, duration, and intensity to accurately classify the text as related to smoking.

Easy Negative Explanation:
The key phrase in this text is "never used illicit drugs." This indicates that the individual has not engaged in the use of illegal substances. The absence of illicit drug use is a significant factor in determining the classification of this text as negative for drug use. Models should look for explicit statements regarding drug use to make accurate classifications in similar cases.


***Save progress***

In [7]:
# Load your saved progress
save_data = {
    'hard_examples': hard_examples,
    'easy_positive': easy_positive,
    'easy_negative': easy_negative,
    'test_df': test_df,
    'results_50': results_50,
    'categories': categories
}
with open('procgress_checkpoint_step61.pkl', 'rb') as f:
    save_data = pickle.load(f)

# Extract the variables
hard_examples = save_data['hard_examples']
easy_positive = save_data['easy_positive']
easy_negative = save_data['easy_negative']
test_df = save_data['test_df']
results_50 = save_data['results_50']
categories = save_data['categories']
# Verify what we loaded
print("Loaded checkpoint!")
print(f"✓ Hard examples: {len(hard_examples)}")
print(f"✓ Easy positive: {easy_positive['premise'][:50]}...")
print(f"✓ Easy negative: {easy_negative['premise'][:50]}...")
print(f"✓ Test set size: {len(test_df)}")
print()

# Package easy examples as tuple for the function
easy_examples = (easy_positive, easy_negative)

Loaded checkpoint!
✓ Hard examples: 2
✓ Easy positive: The patient smokes about a pack a day for more tha...
✓ Easy negative: She has never used illicit drugs...
✓ Test set size: 200



In [ ]:
# =================================================================================
# Step 4: Create the Full Evaluation Function 
# Now let's put it all together and evaluate all 5 configurations on your test set:
# =================================================================================
def evaluate_all_configurations(test_df, easy_examples, hard_examples, 
                                categories, deployment_name="gpt-35-turbo"):
    """
    Evaluate ALL 5 configurations:
    1. 0-Shot
    2. 2-Shot Easy (no explanation)
    3. 2-Shot Easy (with explanation)
    4. 2-Shot Hard (no explanation)
    5. 2-Shot Hard (with explanation)
    """
    from llm_utils import run_llm_annotation, parse_llm_labels
    
    results = {}
    easy_positive, easy_negative = easy_examples
    
    print("="*70)
    print("STARTING 5-CONFIGURATION EVALUATION")
    print(f"Test samples: {len(test_df)}")
    print("="*70)
    
    # Config 1: 0-Shot
    print("\n[1/5] Running 0-Shot...")
    preds_0shot = run_llm_annotation(
        df=test_df,
        categories=categories,
        examples="",
        deployment_name=deployment_name,
        sleep_time=1
    )
    results['0-Shot'] = preds_0shot
    print("✓ 0-Shot complete")
    
    # Config 2: 2-Shot Easy (no exp)
    print("\n[2/5] Running 2-Shot Easy (no explanations)...")
    examples_easy_no_exp = f"""
Example 1 (Positive):
Premise: "{easy_positive['premise']}"
Answer: {parse_llm_labels(easy_positive['gold_labels'])}

Example 2 (Negative):
Premise: "{easy_negative['premise']}"
Answer: {parse_llm_labels(easy_negative['gold_labels'])}
"""
    preds_2easy_no_exp = run_llm_annotation(
        df=test_df,
        categories=categories,
        examples=examples_easy_no_exp,
        deployment_name=deployment_name,
        sleep_time=1
    )
    results['2-Shot Easy'] = preds_2easy_no_exp
    print("✓ 2-Shot Easy complete")
    
    # Config 3: 2-Shot Easy (with exp)
    print("\n[3/5] Running 2-Shot Easy (with explanations)...")
    examples_easy_with_exp = f"""
Example 1 (Positive):
Premise: "{easy_positive['premise']}"
Answer: {parse_llm_labels(easy_positive['gold_labels'])}
Explanation: {easy_positive['explanation']}

Example 2 (Negative):
Premise: "{easy_negative['premise']}"
Answer: {parse_llm_labels(easy_negative['gold_labels'])}
Explanation: {easy_negative['explanation']}
"""
    preds_2easy_with_exp = run_llm_annotation(
        df=test_df,
        categories=categories,
        examples=examples_easy_with_exp,
        deployment_name=deployment_name,
        sleep_time=1
    )
    results['2-Shot Easy + Exp'] = preds_2easy_with_exp
    print("✓ 2-Shot Easy + Exp complete")
    
    # Config 4: 2-Shot Hard (no exp)
    print("\n[4/5] Running 2-Shot Hard (no explanations)...")
    examples_hard_no_exp = f"""
Example 1 (Negative):
Premise: "{hard_examples[0]['premise']}"
Answer: {parse_llm_labels(hard_examples[0]['gold_labels'])}

Example 2 (Positive):
Premise: "{hard_examples[1]['premise']}"
Answer: {parse_llm_labels(hard_examples[1]['gold_labels'])}
"""
    preds_2hard_no_exp = run_llm_annotation(
        df=test_df,
        categories=categories,
        examples=examples_hard_no_exp,
        deployment_name=deployment_name,
        sleep_time=1
    )
    results['2-Shot Hard'] = preds_2hard_no_exp
    print("✓ 2-Shot Hard complete")
    
    # Config 5: 2-Shot Hard (with exp)
    print("\n[5/5] Running 2-Shot Hard (with explanations)...")
    examples_hard_with_exp = f"""
Example 1 (Negative):
Premise: "{hard_examples[0]['premise']}"
Answer: {parse_llm_labels(hard_examples[0]['gold_labels'])}
Explanation: {hard_examples[0]['explanation']}

Example 2 (Positive):
Premise: "{hard_examples[1]['premise']}"
Answer: {parse_llm_labels(hard_examples[1]['gold_labels'])}
Explanation: {hard_examples[1]['explanation']}
"""
    preds_2hard_with_exp = run_llm_annotation(
        df=test_df,
        categories=categories,
        examples=examples_hard_with_exp,
        deployment_name=deployment_name,
        sleep_time=1
    )
    results['2-Shot Hard + Exp'] = preds_2hard_with_exp
    print("✓ 2-Shot Hard + Exp complete")
    
    print("\n" + "="*70)
    print("ALL 5 CONFIGURATIONS COMPLETE! 🎉")
    print("="*70)
    
    return results

In [ ]:
# ===========================Sanity Check=====================================v
# Create a small test set (10 samples)
test_df_small = test_df.sample(n=10, random_state=42)

print("Testing with 10 samples first...")
print(f"Small test set size: {len(test_df_small)}")
print()

# Run the evaluation on small test set
test_results = evaluate_all_configurations(
    test_df=test_df_small,
    easy_examples=easy_examples,
    hard_examples=hard_examples,
    categories=categories,
    deployment_name="gpt-35-turbo"
)

# Quick check of results
print("\n" + "="*70)
print("QUICK RESULTS CHECK")
print("="*70)
for config_name, predictions in test_results.items():
    print(f"{config_name}: {len(predictions)} predictions")
    print(f"  Sample: {predictions[0][:80]}...")
    print()

Testing with 10 samples first...
Small test set size: 10

STARTING 5-CONFIGURATION EVALUATION
Test samples: 10

[1/5] Running 0-Shot...
✓ 0-Shot complete

[2/5] Running 2-Shot Easy (no explanations)...
✓ 2-Shot Easy complete

[3/5] Running 2-Shot Easy (with explanations)...
✓ 2-Shot Easy + Exp complete

[4/5] Running 2-Shot Hard (no explanations)...
✓ 2-Shot Hard complete

[5/5] Running 2-Shot Hard (with explanations)...
✓ 2-Shot Hard + Exp complete

ALL 5 CONFIGURATIONS COMPLETE! 🎉

QUICK RESULTS CHECK
0-Shot: 10 predictions
  Sample: []...

2-Shot Easy: 10 predictions
  Sample: []...

2-Shot Easy + Exp: 10 predictions
  Sample: []...

2-Shot Hard: 10 predictions
  Sample: []...

2-Shot Hard + Exp: 10 predictions
  Sample: []...



In [31]:
# run 200 test samples
print("="*70)
print("RUNNING FULL EVALUATION ON 200 SAMPLES")
print("Estimated time: 15-20 minutes")
print("Estimated cost: ~$2-3")
print("="*70)
print()
# Run on full 200 samples
full_results = evaluate_all_configurations(
    test_df=test_df,  # Use the full 200-sample test set
    easy_examples=easy_examples,
    hard_examples=hard_examples,
    categories=categories,
    deployment_name="gpt-35-turbo"
)
import pickle
with open('full_evaluation_results.pkl', 'wb') as f:
    pickle.dump(full_results, f)

print("\n✅ Results saved to 'full_evaluation_results.pkl'")

RUNNING FULL EVALUATION ON 200 SAMPLES
Estimated time: 15-20 minutes
Estimated cost: ~$2-3

STARTING 5-CONFIGURATION EVALUATION
Test samples: 200

[1/5] Running 0-Shot...
✓ 0-Shot complete

[2/5] Running 2-Shot Easy (no explanations)...
✓ 2-Shot Easy complete

[3/5] Running 2-Shot Easy (with explanations)...
✓ 2-Shot Easy + Exp complete

[4/5] Running 2-Shot Hard (no explanations)...
✓ 2-Shot Hard complete

[5/5] Running 2-Shot Hard (with explanations)...
✓ 2-Shot Hard + Exp complete

ALL 5 CONFIGURATIONS COMPLETE! 🎉

✅ Results saved to 'full_evaluation_results.pkl'


***Save Progress***

In [23]:
# Load full evaluation results
with open('full_evaluation_results.pkl', 'rb') as f:
    full_results = pickle.load(f)   
print("Loaded full evaluation results!")
print(f"Configurations evaluated: {list(full_results.keys())}")
print(f"Samples per configuration: {len(full_results['0-Shot'])}")

# sanity check
for config_name, predictions in full_results.items():
    print(f"{config_name}:")
    print(f"  Total predictions: {len(predictions)}")
    print(f"  Sample prediction: {predictions[0]}")

Loaded full evaluation results!
Configurations evaluated: ['0-Shot', '2-Shot Easy', '2-Shot Easy + Exp', '2-Shot Hard', '2-Shot Hard + Exp']
Samples per configuration: 200
0-Shot:
  Total predictions: 200
  Sample prediction: ["employment"]
2-Shot Easy:
  Total predictions: 200
  Sample prediction: []
2-Shot Easy + Exp:
  Total predictions: 200
  Sample prediction: ["employment"]
2-Shot Hard:
  Total predictions: 200
  Sample prediction: []
2-Shot Hard + Exp:
  Total predictions: 200
  Sample prediction: []


In [ ]:
# =================================================================================
# Step 5: parse predictions and prepare for metrics
# =================================================================================
# prepare variables for gpt4.0 prediction
parsed_results={}
for config_name, raw_predictions in full_results.items():
    parsed_results[config_name]=[
        parse_llm_labels(pred) for pred in raw_predictions
    ]
    print(f"✓ Parsed {config_name}")
print()

# Parse gold labels from test_df
gold_labels_parsed=[
    parse_llm_labels(label) for label in test_df['gold_labels']
]
print(f"✓ Parsed {len(gold_labels_parsed)} gold labels")
# Quick verification - show a few examples
print("Sample comparison")
for i in range(3):
    print(f"\nExample {i+1}:")
    print(f"   Gold: {gold_labels_parsed[i]}")
    print(f"   0-shot: {parsed_results['0-Shot'][i]}")
    print(f"   2-shot Hard+Exp: {parsed_results['2-Shot Hard + Exp'][i]}")

✓ Parsed 0-Shot
✓ Parsed 2-Shot Easy
✓ Parsed 2-Shot Easy + Exp
✓ Parsed 2-Shot Hard
✓ Parsed 2-Shot Hard + Exp

✓ Parsed 200 gold labels
Sample comparison

Example 1:
   Gold: []
   0-shot: ['employment']
   2-shot Hard+Exp: []

Example 2:
   Gold: ['alcohol']
   0-shot: ['alcohol']
   2-shot Hard+Exp: ['alcohol']

Example 3:
   Gold: ['smoking']
   0-shot: ['smoking']
   2-shot Hard+Exp: ['smoking']


In [ ]:
# Calculate metrics for all 5 configurations
print("="*70)
print("CALCULATING METRICS FOR ALL CONFIGURATIONS")
print("="*70)
print()

all_metrics = {}

for config_name in ['0-Shot', '2-Shot Easy', '2-Shot Easy + Exp', 
                     '2-Shot Hard', '2-Shot Hard + Exp']:
    print(f"Calculating metrics for {config_name}...")
    
    metrics = calculate_multilabel_metrics(
        gold_labels=gold_labels_parsed,
        predicted_labels=parsed_results[config_name],
        categories=categories
    )
    
    all_metrics[config_name] = metrics
    
    # Print summary
    print(f"  F1 (micro): {metrics['f1_micro']:.3f}")
    print(f"  F1 (macro): {metrics['f1_macro']:.3f}")
    print(f"  Cohen's Kappa: {metrics['cohen_kappa']:.3f}")
    print(f"  Exact Match: {metrics['exact_match_accuracy']:.3f}")
    print()

print("="*70)
print("METRICS CALCULATION COMPLETE!")
print("="*70)

CALCULATING METRICS FOR ALL CONFIGURATIONS

Calculating metrics for 0-Shot...
  F1 (micro): 0.576
  F1 (macro): 0.578
  Cohen's Kappa: 0.549
  Exact Match: 0.535

Calculating metrics for 2-Shot Easy...
  F1 (micro): 0.845
  F1 (macro): 0.685
  Cohen's Kappa: 0.837
  Exact Match: 0.865

Calculating metrics for 2-Shot Easy + Exp...
  F1 (micro): 0.827
  F1 (macro): 0.693
  Cohen's Kappa: 0.818
  Exact Match: 0.845

Calculating metrics for 2-Shot Hard...
  F1 (micro): 0.802
  F1 (macro): 0.680
  Cohen's Kappa: 0.792
  Exact Match: 0.820

Calculating metrics for 2-Shot Hard + Exp...
  F1 (micro): 0.786
  F1 (macro): 0.681
  Cohen's Kappa: 0.775
  Exact Match: 0.800

METRICS CALCULATION COMPLETE!


In [ ]:
# save all_metrics
with open('all_metrics.pkl', 'wb') as f:
    pickle.dump(all_metrics, f)

1. SURPRISING: Easy Examples > Hard Examples!
This is DIFFERENT from the paper!
The paper found: Hard + Exp dominates; My results: Easy (no exp) dominates
Why?

My hard examples were BOTH about negations (score=3)
Maybe too specific? Only taught negation handling
Easy examples were more diverse (positive smoking case + negative empty case)
More generalizable teaching!
2. Explanations HURT Performance!
**This matches the paper's "over-regularization" finding!**
- Explanations make the model more conservative
- Slightly reduces performance
**3. ALL 2-Shot Configs Beat 0-Shot Significantly!**
**Improvement:** +20-27% F1 score! 🚀

**4. Cohen's Kappa Interpretation:**

According to the paper and literature:
- **0.41-0.60:** Moderate agreement
- **0.61-0.80:** Substantial agreement  
- **0.81-1.00:** Almost perfect agreement

**My results:**
- 0-Shot: 0.549 (Moderate) ⚠️
- 2-Shot Easy: **0.837 (Almost perfect!)** ✅
- Paper reported: 0.82-0.92 for best configs ✅
Why Easy Beat Hard (My Theory):

**Your Hard Examples:**
```
1. "Does not smoke, drink..." → [] (negation)
2. "Uses no tobacco, alcohol..." → [] (negation)
```
- Both teach: "Negations → empty"
- Too narrow! Only fixes one error type

**Your Easy Examples:**
```
1. "Smokes a pack a day..." → ['smoking'] (clear positive)
2. "Never used illicit drugs" → [] (clear negative)
Teaches BOTH: When to predict AND when not to predict
More balanced training!

🎯 Conclusion:
Your experiment actually reveals something important:
✅ 2-shot learning works (huge improvement over 0-shot)
✅ Cohen's Kappa 0.837 matches paper quality (0.82-0.92)
⚠️ Hard examples need diversity - yours were too focused on negations
⚠️ Explanations help consistency but may over-regularize

In [ ]:
#=================================================================================
# Create 4-shot examples to improve performance
#=================================================================================
def create_4shot_prompt(easy_examples, hard_examples):
    easy_pos, easy_neg = easy_examples
    
    examples_4shot = f"""
Example 1 (Easy Positive):
Premise: "{easy_pos['premise']}"
Answer: {parse_llm_labels(easy_pos['gold_labels'])}

Example 2 (Easy Negative):
Premise: "{easy_neg['premise']}"
Answer: {parse_llm_labels(easy_neg['gold_labels'])}

Example 3 (Hard Negative - Tricky):
Premise: "{hard_examples[0]['premise']}"
Answer: {parse_llm_labels(hard_examples[0]['gold_labels'])}
Explanation: {hard_examples[0]['explanation']}

Example 4 (Hard Positive - Tricky):
Premise: "{hard_examples[1]['premise']}"
Answer: {parse_llm_labels(hard_examples[1]['gold_labels'])}
Explanation: {hard_examples[1]['explanation']}
"""
    return examples_4shot

# Test on 10 samples first
print("Testing 4-Shot on 10 samples...")
test_10 = test_df.sample(n=10, random_state=99)

examples_4shot = create_4shot_prompt(easy_examples, hard_examples)
preds_4shot = run_llm_annotation(
    df=test_10,
    categories=categories,
    examples=examples_4shot,
    deployment_name="gpt-35-turbo",
    sleep_time=1
)

print(f"Got {len(preds_4shot)} predictions!")

Testing 4-Shot on 10 samples...
Got 10 predictions!


In [16]:
print("="*70)
print("RUNNING 4-SHOT EVALUATION ON 200 SAMPLES")
print("="*70)

# Run on full test set
examples_4shot = create_4shot_prompt(easy_examples, hard_examples)
preds_4shot_full = run_llm_annotation(
    df=test_df,
    categories=categories,
    examples=examples_4shot,
    deployment_name="gpt-35-turbo",
    sleep_time=1
)

# Parse predictions
parsed_4shot = [parse_llm_labels(pred) for pred in preds_4shot_full]

# Calculate metrics
metrics_4shot = calculate_multilabel_metrics(
    gold_labels=gold_labels_parsed,
    predicted_labels=parsed_4shot,
    categories=categories
)

print("\n4-Shot Results:")
print(f"  F1 (micro): {metrics_4shot['f1_micro']:.3f}")
print(f"  F1 (macro): {metrics_4shot['f1_macro']:.3f}")
print(f"  Cohen's Kappa: {metrics_4shot['cohen_kappa']:.3f}")
print(f"  Exact Match: {metrics_4shot['exact_match_accuracy']:.3f}")

print("\nComparison:")
print(f"  2-Shot Easy: F1={all_metrics['2-Shot Easy']['f1_micro']:.3f}")
print(f"  4-Shot:      F1={metrics_4shot['f1_micro']:.3f}")
print(f"  Improvement: {(metrics_4shot['f1_micro'] - all_metrics['2-Shot Easy']['f1_micro'])*100:+.1f}%")

RUNNING 4-SHOT EVALUATION ON 200 SAMPLES

4-Shot Results:
  F1 (micro): 0.815
  F1 (macro): 0.668
  Cohen's Kappa: 0.806
  Exact Match: 0.830

Comparison:
  2-Shot Easy: F1=0.845
  4-Shot:      F1=0.815
  Improvement: -3.0%
